<a href="https://colab.research.google.com/github/fgith/data-science/blob/main/kaggle/heart_disease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import pandas as pd
import matplotlib.pyplot as plt

In [28]:
! wget "https://raw.githubusercontent.com/fgith/data-science/main/datasets/healthcare-dataset-stroke-data.csv"

--2022-05-15 12:10:15--  https://raw.githubusercontent.com/fgith/data-science/main/datasets/healthcare-dataset-stroke-data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 316971 (310K) [text/plain]
Saving to: ‘healthcare-dataset-stroke-data.csv.2’

healthcare-dataset- 100%[===================>] 309.54K  --.-KB/s    in 0.02s   

2022-05-15 12:10:16 (12.3 MB/s) - ‘healthcare-dataset-stroke-data.csv.2’ saved [316971/316971]



# **1. Load Data**

In [29]:
df = pd.read_csv("healthcare-dataset-stroke-data.csv")
df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


# **2. Explore Data**

In [70]:
print('--Data set shape--')
print(df.shape)
print('------------------------------')
print('--Column-wise unique values--')
cols = ['gender','hypertension','ever_married','work_type','Residence_type','smoking_status','stroke','heart_disease']
for i in range(len(cols)):
  print(cols[i],": ",df[cols[i]].unique())
print('------------------------------')
print('--Column-wise NULL values--')
print(df.isna().sum())
print('------------------------------')

--Data set shape--
(5110, 12)
------------------------------
--Column-wise unique values--
gender :  ['Male' 'Female' 'Other']
hypertension :  [0 1]
ever_married :  ['Yes' 'No']
work_type :  ['Private' 'Self-employed' 'Govt_job' 'children' 'Never_worked']
Residence_type :  ['Urban' 'Rural']
smoking_status :  ['formerly smoked' 'never smoked' 'smokes' 'Unknown']
stroke :  [1 0]
heart_disease :  [1 0]
------------------------------
--Column-wise NULL values--
id                     0
gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  201
smoking_status         0
stroke                 0
dtype: int64
------------------------------
